In [9]:
%matplotlib inline
import pyspeckit
import numpy as np
from astropy import units as u
from astropy.io import fits
from astropy import constants as c
import matplotlib.pyplot as plt
from astropy.convolution import convolve, Box1DKernel
from c_normalize import c_normalize
import glob
import pandas as pd
import warnings
import mplcursors
import yaml
pd.options.display.max_rows = 200
warnings.filterwarnings("ignore")
plt.style.use('seaborn-paper')

In [4]:
from contextlib import contextmanager
import sys, os

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

## Notebook for Outlining Batch EQW Fitting Workflow applied in pycalc_ew.py

In [10]:
# Select Datasets and extract equivalent widths
spectrum = glob.glob("../data/ph_ctrl_stars/tame_inputs/BD*wavsoln.fits")
lc_range = np.arange(0.95,0.995,0.0001)
len_range = 10


d = {'Spectrum File': spectrum}
disp_df = pd.DataFrame(data=d)
disp_df

with open('line_pars.yml') as file:
    pars_dict = yaml.load(file, Loader=yaml.FullLoader)

In [ ]:
width = 10 #Distance from the line on both sides to sample the local continuum from
gauss_amp = -0.2
gauss_width = 0.15
gauss_centhresh = 0.2

survey = []
all_meas = []
for line in lines:
    # Set fitting parameters
    multicomponent=False
    if str(line) in pars_dict.keys():
        key = str(line)
        print(key)
        if len(pars_dict[key][3]) > 1:
            multicomponent=True 
        
        width = pars_dict[key][0] #Distance from the line on both sides to sample the local continuum from
        gauss_centhresh_l = pars_dict[key][1]
        gauss_centhresh_r = pars_dict[key][2]  
        gauss_amps = pars_dict[key][3] #list for components
        gauss_cenoffs = pars_dict[key][4] #list for components
        gauss_width = pars_dict[key][5]
        c_select = pars_dict[key][6]


    else:
        width = 10 #Distance from the line on both sides to sample the local continuum from
        gauss_amps = [-0.2]
        gauss_width = 0.15
        gauss_centhresh_l = 0.2
        gauss_centhresh_r = 0.2
        gauss_cenoffs = [0.0]
        c_select = 0

    percent_difference = True
    diff = []
    for row in disp_df.iterrows():
        spec_file = row[1]
        if (str(line) == '7189.16') and ('ngc2204' in spec_file):
            continue

        s_hdu = fits.open(spec_file)
    
        # Grab the flux and wavelength arrays from the spectrum
        s_data = s_hdu[1].data
        s_flux = s_data['FLUX']
        #smoothed_flux = convolve(s_flux, Box1DKernel(5)) # Smooth the flux for more reliable continuum fitting
        #s_flux = smoothed_flux
        s_wav = s_data['WAVEL']

        # Specify up a specific line to measure the EQW for, and specify the wavelength range to sample for the local continuum
        lim_l = line - width
        lim_r = line + width

        # Mask the flux and wavelength arrays based on the sampled wavelength range
        wav_mask = (s_wav > lim_l) & (s_wav < lim_r)
        s_flux = s_flux[wav_mask]
        s_wav = s_wav[wav_mask]

        # Normalize the spectrum with the local continuum to 1.0, c_normalize returns the normalized spectrum in norm, but also the local continuum fit in yfit if desired
        yfit, norm, _ = c_normalize(s_flux, s_wav, median_replace=False, cheby=True, low_cut = 0.99)
        # Load the normalized spectrum into a pyspeckit.Spectrum object
        sp = pyspeckit.Spectrum(data=norm, xarr=s_wav * u.AA)

        #Get around pyspeckits reliance on a generated baseline and just generate basespec as a ones array (continuum is always one in normalized case)
        sp.baseline.basespec = np.ones(len(s_wav))
        

        #Fit a gaussian to the line, this may be too simplistic for blended lines
        guesses = []
        for amp,cenoff in zip(gauss_amps,gauss_cenoffs):
            guesses.append(amp)
            guesses.append(line+cenoff)
            guesses.append(gauss_width)
        #print(guesses)
            
        with suppress_stdout(): #Suppress some annoying info messages that don't translate well to automation
            sp.specfit(fittype='gaussian', guesses = guesses, exclude= [0,line-gauss_centhresh_l, line+gauss_centhresh_r, line+5000])
        fwhm = sp.specfit.parinfo[2].value

        #Measure the Equivalent Width of the gaussian line fit against the normalized baseline
        EQW = sp.specfit.EQW(plot=False, continuum_as_baseline=True, xmin = 0, xmax = len(norm),components=multicomponent)
        if multicomponent:
            EQW = EQW[c_select]*1000 #mA
        else:
            EQW = EQW * 1000 #mA
        all_meas.append([line, spec_file.split('/')[-1], ((EQW-byhand_eqw)/byhand_eqw) * 100,EQW-byhand_eqw, fwhm]) #append both formats to the all measurements df
        if percent_difference:
            ew_diff = ((EQW-byhand_eqw)/byhand_eqw) * 100 #Percent Difference
        else:
            ew_diff = EQW-byhand_eqw #mA Difference
        if ew_diff < 1000: #Try to cut out some of these crazy fits
            diff.append(ew_diff)
    if percent_difference:
        symbol = "%"
    else:
        symbol = " mA"
    line_stats = [line, np.median(diff),np.std(diff)]
    survey.append(line_stats)

survey_df = pd.DataFrame(survey, columns=["Line (Angstroms)", "Median Difference","Standard Deviation of Difference"])
all_meas_df = pd.DataFrame(all_meas, columns=["Line (Angstroms)", "Spectrum", "Difference (%)","Difference (mA)","FWHM"])